### Import modules

In [79]:
# konlpy: 한국어 처리를 위한 자연어 처리 라이브러리를 설치
!pip install konlpy

In [80]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.utils import to_categorical
from konlpy.tag import Okt
from sklearn.preprocessing import LabelEncoder

### DATA Preprocessing

In [81]:
df = pd.read_csv('KDI 시사용어사전.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7771 entries, 0 to 7770
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   용어      7771 non-null   object
 1   설명      7451 non-null   object
dtypes: object(2)
memory usage: 121.5+ KB


In [82]:
df = df.dropna()

In [83]:
# 용어별로 빈도수를 계산하고, 빈도수가 10 이상인 용어만 선택
count = df['용어'].value_counts()
over10 = list(count[count >= 10].index)

In [84]:
df = df[df['용어'].isin(over10)]
df

,용어,설명
36,가산금리,기준금리에 신용도 등의 조건에 따라 덧붙이는 금리를 가산금리(스프레드: spread...
37,가산금리,"즉, 대출 등의 금리를 정할 때 기준금리에 덧붙이는 위험가중 금리를 말한다"
38,가산금리,"따라서 신용도가 높아 위험이 적으면 가산금리가 낮아지고, 반대로 신용도가 낮아 위험..."
39,가산금리,실제로 우리나라는 1997년 외환위기시 부족했던 달러화를 해외에서 차입하거나 외평채...
40,가산금리,이는 우리나라가 외환위기로 인해 대외신인도가 그만큼 낮아진데 따른 것이다
...,...,...
7683,황의법칙,그 내용은 반도체의 집적도가 2배로 증가하는 시간이 1년으로 단축되었으며 무어의 법...
7684,황의법칙,그리고 이를 주도하는 것은 모바일 기기와 디지털 가전제품 등 non-PC분야라고 하였다
7685,황의법칙,이 규칙을 황창규 사장의 성을 따서 ‘황의 법칙’이라고 한다
7686,황의법칙,"실제 삼성전자는 1999년에 256M 낸드플래시메모리를 개발하고, 2000년 512..."


### Text Tokenizing

In [85]:
# Okt 객체를 사용해 텍스트 데이터를 형태소 단위로 분리
okt = Okt()

In [86]:
def tokenize(text):
    return okt.morphs(text)

In [87]:
df['tokens'] = df['설명'].apply(tokenize)

### Token Indexing

In [88]:
# 모든 토큰을 인덱싱하여 각 토큰에 고유 번호를 할당
all_tokens = [token for sublist in df['tokens'] for token in sublist]
unique_tokens = list(set(all_tokens))
token_index = {token: i+1 for i, token in enumerate(unique_tokens)}

### 시퀀스 데이터 생성 및 패딩

In [89]:
# 토큰화된 데이터를 시퀀스 형태로 변환하고 패딩을 추가하여 길이를 일정하게 맞춤.
sequences = df['tokens'].apply(lambda x: [token_index[token] for token in x])

In [90]:
max_len = max(len(seq) for seq in sequences)
data = pad_sequences(sequences.tolist(), maxlen=max_len, padding='post')

### 라벨 인코딩 및 원-핫 인코딩

In [91]:
# 용어를 숫자 라벨로 변환하고 원-핫 인코딩을 적용
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df['용어'])
labels = to_categorical(integer_encoded)

### 데이터 분할

In [92]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)

### LSTM 모델 구성 및 훈련

In [93]:
# 양방향 LSTM 모델을 구성하고 학습
model = Sequential()
model.add(Embedding(input_dim=len(token_index) + 1, output_dim=256, input_length=max_len))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(len(set(df['용어'])), activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.25)

Epoch 1/15
22/22 [==============================] - 46s 1s/step - loss: 4.6877 - accuracy: 0.0114 - val_loss: 4.6801 - val_accuracy: 0.0085
Epoch 2/15
22/22 [==============================] - 26s 1s/step - loss: 4.6290 - accuracy: 0.0400 - val_loss: 4.6422 - val_accuracy: 0.0128
Epoch 3/15
22/22 [==============================] - 26s 1s/step - loss: 4.2924 - accuracy: 0.1000 - val_loss: 4.4378 - val_accuracy: 0.0556
Epoch 4/15
22/22 [==============================] - 24s 1s/step - loss: 3.7397 - accuracy: 0.2100 - val_loss: 4.3351 - val_accuracy: 0.0897
Epoch 5/15
22/22 [==============================] - 23s 1s/step - loss: 3.2672 - accuracy: 0.2943 - val_loss: 4.1452 - val_accuracy: 0.1282
Epoch 6/15
22/22 [==============================] - 27s 1s/step - loss: 2.8486 - accuracy: 0.4029 - val_loss: 4.0554 - val_accuracy: 0.1111
Epoch 7/15
22/22 [==============================] - 26s 1s/step - loss: 2.5044 - accuracy: 0.4843 - val_loss: 3.9318 - val_accuracy: 0.1752
Epoch 8/15
22/22 [==

### 모델 평가

In [94]:
# 테스트 데이터셋으로 모델을 평가하고 정확도를 출력
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

10/10 [==============================] - 3s 262ms/step - loss: 3.4823 - accuracy: 0.2340
Test Accuracy: 0.23397435247898102


### 용어 예측 함수

In [95]:
# 주어진 설명을 바탕으로 경제 용어를 예측
def predict_term(description):
    sequence = [token_index.get(token, 0) for token in okt.morphs(description)]
    padded_sequence = pad_sequences([sequence], maxlen=max_len, padding='post')
    prediction = model.predict(padded_sequence)
    predicted_label = np.argmax(prediction, axis=1)
    predicted_term = label_encoder.inverse_transform(predicted_label)[0]
    return predicted_term

In [96]:
# 사용자로부터 입력을 받아 해당 설명에 대한 경제 용어를 예측
def predict_from_user_input():
    while True:
        new_description = input("경제 용어에 대한 설명을 입력하세요 : ")
        if new_description.lower() == '종료':
            break
        predicted_term = predict_term(new_description)
        print(f"입력 내용의 연관 경제 용어는 '{predicted_term}'입니다.\n")
# 종료하려면 '종료' 입력

In [97]:
predict_from_user_input()

경제 용어에 대한 설명을 입력하세요 : 경쟁을 하는데 시장에서 독점적인 위치를 가지고 있습니다
1/1 [==============================] - 2s 2s/step
입력 내용의 연관 경제 용어는 '자연독점'입니다.

경제 용어에 대한 설명을 입력하세요 : 주식회사에서 주주들에게 주식 수만큼 일정 금액을 돌려주는 것입니다
1/1 [==============================] - 0s 101ms/step
입력 내용의 연관 경제 용어는 '증거금'입니다.

경제 용어에 대한 설명을 입력하세요 : 국가에 돈이 부족해
1/1 [==============================] - 0s 80ms/step
입력 내용의 연관 경제 용어는 '그룹웨어'입니다.

경제 용어에 대한 설명을 입력하세요 : 종료
